In [0]:
from pyspark.sql import SparkSession # Puerta de entrada para trabajar con spark <-- SIEMPRE DEBEMOS IMPORTAR LA LLAVE MAESTRA QUE INICIA TODO.
from pyspark.sql.functions import *  # Funciones propias del módulo SQL de Spark, para trabajar sobre Dataframes.
from pyspark.sql.types import *
spark = SparkSession.builder.appName("ApacheSparkDatabricksApuntes").getOrCreate() 
"""
^          ^__________^        ^_________^                               ^
|                |                   |                                   | 
Variable   Constructor de Sesión   Nombre Aplicación       Evita conflicto del SparkSession"""


### RDDs (Versión Databricks Community Edition)

#### Link de Databrick Community (Solo para el ejercicio de RDDs): [Databrick Community](https://community.cloud.databricks.com/?o=2956217321084781)

##### EXPLICACIÓN RDDs

In [0]:

sc = spark.sparkContext # INSTANCIAMOS SPARK CONTEXT PARA EL USO DE RDDs
# Crear RDD desde lista
lista_rdd_final = sc.parallelize([1, 2, 3, 4, 5]) #<-- Usamos .parallelize() para definir SOLO RDDs
print(lista_rdd_final.collect()) #<-- Al imprimir una lista, siempre haremos referencia a collect()

# # DICCIONARIOS EN SPARK - USANDO RDDS --> parallelize()
diccionario = spark.sparkContext.parallelize({"Nombre":"Brayan","Edad":21}.items())
print(diccionario.collect())

# # # SETS EN SPARK - USANDO RDDS --> parallelize()
sets = spark.sparkContext.parallelize({1,2,3,3,4,4,5})
print(sets.collect())

# # # TUPLAS EN SPARK - USANDO RDDS --> parallelize()
tupla = spark.sparkContext.parallelize((1,2,3,4,5,"BRAYAN"))
print(tupla.collect())

In [0]:
# FUNCIONES DE ACCIONES DE RDDs:
# **Tomaremos de referencia la lista definida:
print(lista_rdd_final.collect()) # Retorna todos los elementos del RDD
print(lista_rdd_final.count()) # Retorna la cantidad de elementos del RDD
print(lista_rdd_final.first()) # Retorna el primer elemento del RDD
print(lista_rdd_final.take(3)) # Retorna los N primeros elementos del RDD

# FUNCIONES DE TRANSFORMACIONES DE RDDs:
# ** Tomaremos de referencia la lisa definida:

# *FILTER
print(lista_rdd_final.filter(lambda x:x%2==0).collect()) # Filtrar elementos del RDD basados en una función

# *MAP
print(lista_rdd_final.map(lambda x:x+2).collect()) # Retorna misma cantidad de elementos del RDD, pero, modificados.

# *FLATMAP
texto = spark.sparkContext.parallelize(["Hola como estas?","Espero muy bien"])
print(texto.flatMap(lambda x: x.split(" ")).collect()) # Retorna una cantidad diferente de elementos del RDD y modificados.

In [0]:
# FUNCIONES DE AGRUPACIÓN:            
# * REDUCE
print(lista_rdd_final.reduce(lambda x,y:x+y)) # Función que permite reducir a la mínima expresión.

# * REDUCE BY KEY
ventas = [
    ("Manzana",10),
    ("Pera",15),
    ("Manzana",5),
    ("Platano",50)
]
diccionario_rdd = spark.sparkContext.parallelize(ventas)
print(diccionario_rdd.reduceByKey(lambda x,y:x+y).collect()) # Permite retornar datos agrupados

##### EJERICICIOS RDDs

In [0]:
# EJERCICIOS BÁSICOS DE APACHE SPARK - RDDs

# 1. 
dias_semana = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]
rdd_dias_semana = spark.sparkContext.parallelize(dias_semana)
print(rdd_dias_semana.collect())

# 2.
numeros = [1,2,3,4,5,6,7,8,9,10]
rdd_numeros = spark.sparkContext.parallelize(numeros)
print(rdd_numeros.count())

# 3. 
ciudades = ["Paris","Shangai","Roma","Buenos Aires"]
rdd_ciudades = spark.sparkContext.parallelize(ciudades)
print(rdd_ciudades.first())

# 4.
numeros = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
numeros_rdd = spark.sparkContext.parallelize(numeros)
print(numeros_rdd.take(5))

# 5.
numeros = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
rdd_numeros = spark.sparkContext.parallelize(numeros)
print(rdd_numeros.filter(lambda x:x%2==0).collect())

# 6.
numeros = [1,2,3,4,5]
rdd_numeros = spark.sparkContext.parallelize(numeros)
rdd_cuadrados = rdd_numeros.map(lambda x:x**2)
print(rdd_cuadrados.collect()) 

# 7.
frases = ["Hola mundo", "Spark es genial", "RDDs son poderosos"]
rdd_frases = spark.sparkContext.parallelize(frases)
print(rdd_frases.flatMap(lambda x:x.split(" ")).collect())

# 8.
numeros = [1,2,3,4,5,6,7,8,9,10]
rdd_numeros = spark.sparkContext.parallelize(numeros)
print(rdd_numeros.reduce(lambda x,y:x+y))

# 9.
lista_tuplas = [("a", 1), ("b", 2), ("a", 3), ("b", 4), ("c", 5)]
rdd_lista_tuplas = spark.sparkContext.parallelize(lista_tuplas)
print(rdd_lista_tuplas.reduceByKey(lambda x,y:x+y).collect())

# 10.
frases = ["Hola mundo", "Spark es genial", "RDDs son poderosos"]
rdd_frases = spark.sparkContext.parallelize(frases)
flat_map_frases = rdd_frases.flatMap(lambda x:x.split(" "))
print(f"Cantidad de  palabras mayor a 4 letras: {flat_map_frases.filter(lambda x:len(x)>4).count()}")




### DATAFRAMES (Versión Databricks Free Edition)

#### Link de [Databrick Free Edition](https://dbc-89f542f8-2df6.cloud.databricks.com/?o=758509963140561)

##### DEFINICIÓN DE UN DATAFRAMES EN DATABRICKS

In [0]:
# 1️⃣ DEFINIR DATAFRAMES DESDE CERO EN APACHE SPARK - DATABRICKS:

    #✴️ Parámetros de createDataframe(): data= *Datos en formato de tupla* y schema= *Nombre de las columnas*

datos = [("Brayan",21),("Rafael",22)] # ⬅️ Lista de tuplas (Cada tupla es un registro en el Dataframe)
df = spark.createDataFrame(data=datos,schema=["Nombre","Edad"]) #⬅️ Definimos el Dataframe (spark.createDataframe)
# df.head()  ⬅️ Muestra el primer registro del Dataframe
# df.show()   #⬅️ Muestra todo los registros del Dataframe (Límite de 20 filas)
# df.tail(3)  #⬅️ Muestra los 3 últimos registros del Dataframe en una sola fila.

# 2️⃣ DEFINIR DATAFRAMES EN APACHE SPARK MEDIANTE UN DICCIONARIO:

diccionario = {"Nombre":["Brayan","Rafael"],"Edad":[21,22]} # ⬅️ Diccionario
df_dict = spark.createDataFrame(data=list(zip(*diccionario.values())),schema=list(diccionario.keys()))
"""
    Vamos descomponiendo y explicando todo lo que sucede en data de este dataframe.
    Paso 1. diccionario.values(): Retorna todos los valores asignados a una llave en el diccionario. 
                                  ➡️ ["Brayan","Rafael"];[21,22]
    2. * : Este asterisco desempaqueta cada valor retornado del Paso 1. 
                                  ➡️ "Brayan","Rafael"   ||   21,22
    3. zip(): Unifica los valores desempaquetados del Paso 2. en una tupla.
                                  ➡️ ("Brayan","Rafael"),(21,22)
    4. list(): Convierte cada tupla retornada del Paso 3. y lo asigan como un elemento ed una lista final.
                                  ➡️[("Brayan","Rafael"),(21,22)]
    5. data: El parámetro de createDataFrame() lo toma como valores correctamente organizados para ser parte de los registros del dataframe.                                 
""" 
df_dict.show()

##### OPERACIONES BÁSICAS CON DATAFRAMES EN DATABRICKS

In [0]:
# Definición del dataframe
diccionario = {
  "Nombre":["Pepe","Juan","Pedro","Carlos","Raul"],
  "Edad":[20,25,26,24,21],
  "Nota":[10,12,15,14,10]
}
# En caso los tipos de datos de cada columna sean variados,
# los definimos aparte y luego lo integramos en createDataFrame(schema=**)
from pyspark.sql.types import * ## Importamos esta librería para la creación de estructuras desde cero.
schema_dataframe = StructType([ #⬅️ Método para definir nuevos tipos de estructuras, en este caso, columnas de un DF.
  StructField("Nombre",StringType(),True), #⬅️ Método para definir columna: StructField(NombreColumna,TipoDato,True)
  StructField("Edad",LongType(),True),
  StructField("Nota",LongType(),True)
])
df_example = spark.createDataFrame(data=list(zip(*diccionario.values())),schema=schema_dataframe) 
df_example.show()

#### Operaciones básicas con dataframes
##============== Obtener valor o valores de una columna en un Dataframe
df_example[["Edad"]].show() # Obtenemos el valor de una columna
df_example[["Nombre","Edad"]].show() # Obtenemos valores de varias columnas

##============== Fitrar valores en una columna específica: método ➡️.filter(col(NombreColumna)&|Condición)
# Filtramos los registros con la edad mayor a 21...
df_example.filter(col("Edad")>21).show() # Mediante la función col(NombreColumna), accedemos a la columna y sus filas. 
# Filtrar valores en dos columnas específicas
df_example.filter(
  (col("Edad")==25) & (col("Nombre")=='Juan')
).show() # La condición es separada en tuplas y con un operador lógico.

##============== Agregar nuevas columnas al Dataframe: método ➡️.withColumn(NombreColumna,Condición/Valor)
# Utilizaremos lit para valores literales y/o asignar valores
df_example = df_example.withColumn( #⬅️ Agregamos una sola columna
  "Estado", # Nuevo nombre de la columna
  lit("Activo") # Agregar el mismo valor para la N cantidad de filas
) 
df_example.show()

df_example = df_example.withColumns({ #⬅️ Agregamos varias columnas: método ➡️.withColumns({NombreColumna:Condición/Valor})
  "Estado": # Nuevo nombre de la columna 1
  lit("Activo"), # Agregar el mismo valor para la N cantidad de filas
  "Fecha Registro": # Nuevo nombre de la columna 2
  lit("2025-07-15"), # Agregar el mismo valor para la N cantidad de filas
}) 
df_example.show()

##=============== Agregar nueva columna al Dataframe en base a una condición: método ➡️.withColumn(NombreColumna,Condición)
df_example = df_example.withColumn(
  "Categorid_Edad",                                        # Nombre de nueva columna
  when(col("Edad")<18,lit("Menor"))                        # Primera condición
  .when((col("Edad")>=18) & (col("Edad")<25),lit("Joven")) # Segunda condición
  .otherwise(lit("Adulto"))                                # Última condición
)
df_example.show()

##=============== Seleccionamos una o más columnas del dataframe: método ➡️.select(col(NombreColumna))
df_example = df_example.select(
  col("Nombre"),                   # Seleccionamos columna Nombre
  col("Edad"),                     # Seleccionamos columna Edad
)
df_example.show()

##=============== Renombrar una columna de un DataFrame: 
# ➡️método .withColumnRenamed(existing=NombreColumnaActual,new=NombreNuevoColumna) ||
# ➡️método .withColumnsRenamed({NombreAntiguoExistente1:NombreNuevoColumna1,NombreAntiguoExistente2:NombreNuevoColumna2}) 

df_example = df_example.withColumnRenamed(existing="Nombre",new="Name") #⬅️ Renombra columnas individuales
df_example = df_example.withColumnsRenamed({"Nombre":"Name","Edad":"Age"}) #⬅️ Renombra varias columnas
# df_example = df_example.selectExpr("Nombre AS Name","Edad as Age") 
#⬅️ .selectExpr() renombra la o las columnas, sin embargo, selecciona esas renombradas y omite el resto (No recomendado) 
df_example.show()

##=============== Eliminar columnas de un Dataframe: ➡️ método .drop(col(NombreColumna)) 
df_example = df_example.drop(col("Nota")) # Ingresar nombre de columna a eliminar
df_example.show()

##=============== Ordenar Dataframe por una columna en específico: ➡️ método .sort(col(NombreColumna).asc() || .desc())
df_example.orderBy(col("Edad").desc()).show() # Forma descendente por la columna Edad
df_example.orderBy(col("Edad").asc()).show() # Forma ascendente por la columna Edad




##### VALORES ATÍPICOS Y NO ATÍPICOS (RANGO INTERQUARTIL)

In [0]:
# 🔍 Análisis de valores atípicos usando el rango intercuartil (IQR)
# Dataset utilizado: 🐧 Penguins (cargado previamente como Data Table en Unity Catalog)
# ⚠️ Próximamente explicaré cómo cargar datasets al Unity Catalog en Databricks Free Edition...

df_penguins = spark.sql("SELECT * FROM workspace.exercises.penguins") #📊LENGUAJE SQL¿?....
# df_penguins.show()

# ESTADÍSTICA DESCRIPTIVA BÁSICA DE UN DATASET (PENGUINS)
# df_penguins.describe().show()

# Rango Interquartil (IQR)
# Permite hallar valores atípicos dentro de nuestro dataset.
# 💡Rango interquartil aplicado a la columna body_mass_g
quartiles_body_mass_g = df_penguins.agg(
expr('percentile(body_mass_g,array(0.25))')[0].alias("q1"), # 💡Importante: expr() viene integrado dentro
expr('percentile(body_mass_g,array(0.50))')[0].alias("q2"), #    de las funciones sql en pyspark.
expr('percentile(body_mass_g,array(0.75))')[0].alias("q3"), # 💡 Cada cuartil es una porción del dataset completo
expr('percentile(body_mass_g,array(1.00))')[0].alias("q4")  #    (0.25,0.50,0.75,1) 
)
# quartiles_body_mass_g.show()

q1 = quartiles_body_mass_g[["q1"]].first() # Quartil 1 ➡️ método .first() trae el primer valor de la columna
## print(q1.q1) # Accede al valor de la columna
q3 = quartiles_body_mass_g[["q3"]].first() # Quartil 3 ➡️ método .first() trae el primer valor de la columna
## print(q3.q3)# Accede al valor de la columna
rango_iqr = q3.q3-q1.q1 # Cálculo de rango intercuartil
## print(rango_iqr)
lower_bound_body = q1.q1 - 1.5 *rango_iqr # Límite superior
upper_bound_body = q3.q3 + 1.5 * rango_iqr # Límite inferior
## print(lower_bound_body)
## print(upper_bound_body)
df_penguins_atipicos = df_penguins.filter(  
    (col("body_mass_g")<lower_bound_body) |
    (col("body_mass_g")>upper_bound_body)
)# ⬅️ Filtramos los valores atípicos (fuera de los umbrales calculados en los límites superior e inferior)
## df_penguins_atipicos.show()
df_penguins_normales = df_penguins.filter(
    (col("body_mass_g")>=lower_bound_body) &
    (col("body_mass_g")<=upper_bound_body)
)# ⬅️ Filtramos los valores limpios(dentro de los umbrales calculados en los límites superior e inferior)
df_penguins_normales.show() # Dataset limpio

### Explicación de Catálagos - Esquemas - Volumenes/Tablas

#### JERARQUÍA UNITY CATALOG

In [0]:
Crear el catálago (carpeta o conjunto de carpetas que almacenará los archivos):
>> Usando SQL EDITOR (SINTAXIS): 
    CREATE CATALOG IF NOT EXISTS nombre_catalago;
>> Usando PySpark (SINTAXIS): 
    spark.sql("CREATE CATALOG IF NOT EXISTS nombre_catalago")


In [0]:
** Crear el esquema (agrupación de archivos basado en el objetivo del proyecto):
>> Usando SQL EDITOR (SINTAXIS): 
    CREATE SCHEMA IF NOT EXISTS nombre_catalago.NombreEsquema;
>> Usando PySpark (SINTAXIS): 
    spark.sql("CREATE SCHEMA IF NOT EXISTS nombre_catalago.NombreEsquema;")

In [0]:
** Crear el volumen (contenedores de archivos físicos sin esquema tabular):
>> Usando SQL EDITOR (SINTAXIS): 
    CREATE VOLUME IF NOT EXISTS nombre_catalago.nombre_esquema.NombreVolumen;
>> Usando PySpark (SINTAXIS): 
    spark.sql("CREATE VOLUME IF NOT EXISTS nombre_catalago.nombre_esquema.NombreVolumen;")

#### ALMACENAR OBJETOS EN UNITY CATALOG

In [0]:
# PATH PARA ACCEDER/ALMACENAR OBJETOS EN VOLUMENES (RECOMENDADO POR GOBERNANZA DE UNITY CATALOG): 
SINTAXIS:
"/Volumes/nombre_catalago/nombre_esquema/nombre_volumen" 
EJEMPLO:
"/Volumes/workspace/exercises/volumen_dataframe"